# 6 - Model and hyper parameters choice

The dataset used in this notebook is the same used in the last two practice sessions: you can copy-paste into the current directory or download it again from [here](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html).
Remember:
- you need the kdd.data.gz file
- it is a compressed archive, you have to extract it

---

This session will focus on:
- model choice
- hyper parameters selection

---

In [ ]:
import pandas as pd
import numpy as np
import time

## Load the dataset

In [ ]:
FILENAME = 'datasets/kddcup.data.corrected'

In [ ]:
# feature names obtained from: http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names
header_names = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 
    'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 
    'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 
    'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 
    'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 
    'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 
    'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 
    'dst_host_srv_rerror_rate', 'attack_type'
]

In [ ]:
df = pd.read_csv(FILENAME, header=None, names=header_names, sep=',')

<div class="alert alert-block alert-info">
<b>
IMPORTANT:
    
The cell below reduces the size of the dataframe by sampling from it. 
This is done to work with a smaller amount of data, in case your machine cannot deal with the whole dataset. 
Try to run the notebook without running this cell but, if you have any problems, come back here, uncomment this line and rerun the whole notebook with less data.
</b>

If you still have troubles, there is a smaller version available on the same website.
The file name is *kddcup.data_10_percent.gz*.
</div>

In [ ]:
df = df.sample(frac=0.60)

## Info about the dataset (pt. 1)

This is the same dataset we used in the past sessions, but it is useful to recap here some information about it.

<div class="alert alert-block alert-danger">
<b>Q: Complete the cell below.</b>
</div>

In [ ]:
# print number of rows, using df.shape
print("Number of rows =", )

<div class="alert alert-block alert-danger">
<b>Q: Complete the cell below.</b>
</div>

In [ ]:
# print number of features, using the drop and the shape methods
print("Number of features =", )

In [ ]:
print("Type of the features:")
df.info()

In [ ]:
set_cat_features = (
    set(df.drop('attack_type', axis=1).columns) - set(df.drop('attack_type', axis=1).describe().columns)
)
print("There are %d categorical features, and they are:" % len(set_cat_features))
print(set_cat_features)

## Creating variables containing indeces of each feature type

In [ ]:
col_names = np.array(header_names)

nominal_idx = [1, 2, 3]
binary_idx = [6, 11, 13, 14, 20, 21]
numeric_idx = list(set(range(41)).difference(nominal_idx).difference(binary_idx))

nominal_cols = col_names[nominal_idx].tolist()
binary_cols = col_names[binary_idx].tolist()
numeric_cols = col_names[numeric_idx].tolist()

## Info about the dataset (pt. 2)

In [ ]:
print("Number of different 'protocol_type's =", len(df['protocol_type'].unique()))
print("Number of occurrences of each protocol_type")
df.groupby('protocol_type').size().reset_index().sort_values(0, ascending=False).rename(columns={0:'cnt'})

<div class="alert alert-block alert-danger">
<b>Q: What is the effect of <code>.rename(columns={0:'cnt'})</code>, in the cell above?</b>
</div>

<div class="alert alert-block alert-success">
ANS
</div>

In [ ]:
print("Number of different 'flag's =", len(df['flag'].unique()))
print("Number of occurrences of each flag")
df.groupby('flag').size().reset_index().sort_values(0, ascending=False).rename(columns={0:'cnt'})

In [ ]:
print("Number of different 'service's =", len(df['service'].unique()))
print("Number of occurrences of each service")
df.groupby('service').size().reset_index().sort_values(0, ascending=False).rename(columns={0:'cnt'})

## Mapping each attack type to one category

In [ ]:
# this cell modifies the data in the dataframe, if you run it more than once, you might encounter some errors
df['attack_type'] = df.apply(lambda r: r['attack_type'][:-1], axis=1)

The file *training_attack_types.txt* maps each of the attacks in the original dataset to 1 category.
The file can be found [here](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html), or you can use the same one you used last time.

#### IMPORTANT
Check that there are no empty lines at the end of the file, otherwise the code below might not work correctly.

In [ ]:
from collections import defaultdict

In [ ]:
category = defaultdict(list)
category['benign'].append('normal')

In [ ]:
TRAINING_ATTACK_TYPES_FILENAME = 'datasets/training_attack_types.txt'

In [ ]:
with open(TRAINING_ATTACK_TYPES_FILENAME, 'r') as f:
    for line in f.readlines():
        attack, cat = line.strip().split(' ')
        category[cat].append(attack)

attack_mapping = {v: k for k in category for v in category[k]}

`attack_mapping`, as created above, is a dictionary that maps each attack_type to one category.

In [ ]:
print("There are %d categories" % len(set(attack_mapping.values())))
print("Their names are:", set(attack_mapping.values()))

In [ ]:
# this is the cell that performs the actual mapping
df['attack_category'] = df.apply(lambda r: attack_mapping[r['attack_type']], axis=1)

<div class="alert alert-block alert-info">
This is similar to what was done above. The difference is that here a new column is created (named "attack_category"), which contains, for each row, the category of the attack type (<code>r['attack_type']</code> is used as a key to access the dictionary <code>attack_mapping</code>) 
</div>

In [ ]:
print("Number of occurrences of each category:")
df.groupby('attack_category').size().reset_index().sort_values(0, ascending=False)

<div class="alert alert-block alert-danger">
<b>Q: Looking at the results of the previous cell you should realize that you must be very careful with training a 5-class classifier... Why?</b>
</div>

<div class="alert alert-block alert-success">
ANS
</div>

## Data preparation: dummy variables

We have some categorical variables. Thus, we have to converte them to one-hot encoded variables.

<div class="alert alert-block alert-danger">
<b>Q: Create a new DataFrame encoding the categorical attributes with one hot encoding.</b>
</div>

In [ ]:
# Convert categorical feature into dummy variables with one-hot encoding
df_one_hot = 

## Data preparation: Train-test split

In [ ]:
from sklearn.model_selection import train_test_split

# Split dataset up into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    df_one_hot.drop(['attack_category', 'attack_type'], axis=1), 
    df_one_hot['attack_category'], 
    test_size=0.2
)

## Data preparation: scaling

This cell might take a while to run

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# if it crashes, you might want to try rerunning the notebook on less data (see df.sample at the beginning)
standard_scaler = StandardScaler().fit(X_train[numeric_cols])

X_train[numeric_cols] = standard_scaler.transform(X_train[numeric_cols])
X_test[numeric_cols] = standard_scaler.transform(X_test[numeric_cols])

<div class="alert alert-block alert-danger">
<b>Q: Why is the scaling performed using the X_train variable only?</b>
</div>

<div class="alert alert-block alert-success">
ANS
</div>

## Data preparation: converting label to integers

In [ ]:
y_train_bin = y_train.apply(lambda x: 0 if x is 'benign' else 1)
y_test_bin = y_test.apply(lambda x: 0 if x is 'benign' else 1)

REMEMBER: `.apply` applies the `lambda` function within the `()` to each element of the sequence (in this case a Series).

---

# CLASSIFICATION

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.metrics import (
    accuracy_score, 
    recall_score,
    precision_score,
    confusion_matrix
)

---

# 6.1 - BINARY CLASSIFICATION

Now you have to train a model for performing binary classification (i.e. detecting whether one entry is malicious or not, without caring about the attack_type).

You have to perform cross validation and hyperparameters selections in order to find the best performing model.

## First example of Cross Validation with GridSearch

The following cell performs hyper parameter selection with 5-fold cross validation.
The comments inside this cell should provide enough explanation to let you write the code for performing cross validation on different models and different hyper parameters.

<div class="alert alert-block alert-info">
<b>IMPORTANT</b>: you'll probably notice that some of the combination of parameters take a very long time to run. This is normal and it is due to the fact that, for each possible combination of parameters, the model is trained several times (value set in GridSearchCV). Thus, the elapsed time that you had seen in the previous prac session will get much longer.
    
You might also have some problems due to memory limitations.
As mentioned at the beginning of the notebook, try to sample the original DF or, alternatively, use the 10percent dataset (you can download it from the website).
    
For this session in the classroom, use small lists of parameters and 2-fold or 3-fold cross validation in order to run the code faster.
However, while working at home you have to explore larger choices of hyperparameters and use 5-fold cross validation in order to find better performing models.
</div>

- This line defines the model, in this case a RandomForestClassifier but it could be any classifier (DecisionTreeClassifier, SVC, etc.)

In [ ]:
classifier = RandomForestClassifier()

- this line defines a Pipeline, in this case it is made of only one element and this is what you will need to use for now

In [ ]:
pipe = Pipeline(steps=[('clf', classifier)])

- parameters of the model in the pipeline can be set using `'__'` separated parameter names. In this case, `'clf__n_estimator'` means that the following list is a list of values for the n_estimators attribute of the clf model. Here, it is the number of trees of the RandomForest

In [ ]:
param_grid = {
    'clf__n_estimators': [10, 25, 50],
}

- performs cross validation for each of the parameters set above. `cv=3` means that I perform 3-fold cross validation


In [ ]:
searchRF = GridSearchCV(pipe, param_grid, iid=False, cv=3, verbose=True)
searchRF.fit(X_train, y_train_bin)

- once the training is done, you can show the parameters of the best performing model

In [ ]:
print(searchRF.best_params_)

- and you can also retrieve it in order to perform the final test on the test set

In [ ]:
trainedRF = searchRF.best_estimator_.get_params()['clf']
trainedRF

In [ ]:
y_pred_RF = trainedRF.predict(X_test)

<div class="alert alert-block alert-danger">
<b>Q: Complete the following cell in order to perform the final evaluation of the model on the test set.</b>
</div>

In [ ]:
print("accuracy:", accuracy_score()) 
print("recall:", recall_score())
print("precision:", precision_score())

## Another example, this time with a DecisionTreeClassifier

It is more convenient to write everything in a unique cell, in order to have everything under control.

Here I perform 3-fold cross validation on a DecisionTreeClassifier and explore 3 possible values of `max_depth` and 2 possible values of `max_leaf_nodes`.

In [ ]:
classifier = DecisionTreeClassifier()

pipe = Pipeline(steps=[('clf', classifier)])

# as you can see, you can set lists of value for all the attributes of the model
param_grid = {
    'clf__max_leaf_nodes': [10, None],
    'clf__max_depth': [2, 5, 10],
}

searchDT = GridSearchCV(pipe, param_grid, iid=False, cv=3, verbose=True)
searchDT.fit(X_train, y_train_bin)

print(searchDT.best_params_)

trainedDT = searchDT.best_estimator_.get_params()['clf']
trainedDT

In [ ]:
y_pred_DT = trainedDT.predict(X_test)

<div class="alert alert-block alert-danger">
<b>Q: Complete the following cell in order to perform the final evaluation of the model on the test set.</b>
</div>

In [ ]:
print("accuracy:", accuracy_score()) 
print("recall:", recall_score())
print("precision:", precision_score())

## Now your turn

<div class="alert alert-block alert-danger">
<b>Q: Use the scikit-learn Pipeline and GridSearchCV in order to find the best model and hyperparameter configuration. Fill the template below.</b>
</div>
As mentioned above, keep things simple for this practice session otherwise you won't have time to see the results of the training.

Possible models to explore:
- GaussianNB
- DecisionTreeClassifier
- RandomForestClassifier
- LogisticRegression
- ... but **do** feel free to experiment with other ones as well! 

In [ ]:
classifier = ... # feel free to use any models you want

pipe = Pipeline(steps=[('clf', classifier)])

param_grid = {
    ...
}

search = GridSearchCV(pipe, param_grid, iid=False, cv=..., verbose=True)
search.fit(X_train, y_train_bin)

print(searchDT.best_params_)

trained_clf = searchDT...
trained_clf

In [ ]:
y_pred = trained_clf...

In [ ]:
print("accuracy:", ...) 
print("recall:", ...)
print("precision:", ...)

---

# 6.2 - 5-CLASS CLASSIFICATION

This last section focuses on performing multiclass classification.
Instead of trying to classify each entry as *attack* or *not attack*, we now try to classify it as belonging to one of the following classes:
- dos
- benign
- probe
- r2l
- u2r

Here is an example of model and hyperparameters selection:

In [ ]:
pipe = Pipeline(steps=[('clf', RandomForestClassifier())])

param_grid = {
    'clf__n_estimators': [50, 100],
    'clf__max_depth': [2, 5, 10],
}

search = GridSearchCV(pipe, param_grid, iid=False, cv=3, verbose=True)
search.fit(X_train, y_train)

print(search.best_params_)

trained_clf = search.best_estimator_.get_params()['clf']
trained_clf

In [ ]:
y_pred = trained_clf.predict(X_test)

In [ ]:
print("accuracy:", accuracy_score(y_test, y_pred)) 

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
clf_report = classification_report(y_test, y_pred, output_dict=True)

In [ ]:
for attack_type in set(attack_mapping.values()):
    print(attack_type)
    print(clf_report[attack_type])
    print()

<div class="alert alert-block alert-danger">
<b>Q: Now it's your turn. Experiment with different models, hyperparameters and try to find the best performing model using cross validation with Pipeline and GridSearchCV. Do not forget to tinker with different features as well: as introduced in the last session, try to remove some features, focus only on some features, etc. in order to improve the classification accuracy.
    
Remember: *one perfect model* to solve this problem does not exist, but by approaching the problem in the correct way, you can get better and better models.
</b>
</div>

Remember, when performing the final evaluation, to look at all the 5 classes and not only at the overall accuracy!

In [ ]:
classifier = ... # feel free to use any models you want

pipe = Pipeline(steps=[('clf', classifier)])

param_grid = {
    ...
}

search = GridSearchCV(pipe, param_grid, iid=False, cv=..., verbose=True)
search.fit(X_train, y_train)

print(searchDT.best_params_)

trained_clf = searchDT...
trained_clf

In [ ]:
y_pred = trained_clf...

In [ ]:
clf_report = 

---

<div class="alert alert-block alert-info">
After you finish the notebook, take a look at this link: https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html

In the second Section ("Face recognition with eigenfaces"), it presents an example of how to use the techniques that you have seen so far in a more challenging problem (face recognition), which is also very relevant from a security perspective, since face recognition is often used for authentication on hand-held devices.
</div>